# Optimization - fine tuning

[1. Optimize LSH](#1-optimize-lsh)

[2. Optimize Hypercube](#2-optimize-hypercube)

[3. Optimize GNNS](#3-optimize-gnns)

[4. Optimize MRNG]()

# Import libraries

In [1]:
import optuna
from optuna.visualization import plot_pareto_front, plot_optimization_history, plot_slice

import pandas

from params import lsh_test, hypercube_test, gnn_test, mrng_test

/home/thantrisp/miniconda3/envs/testpy/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
input_path = b'../MNIST/input.dat'
query_path = b'../MNIST/query.dat'

n = 60000

# 1. Optimize LSH

In [3]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False]),
                 }
    
    print("Trial parameters:", param_dict)

    average_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=100, **param_dict, N=60)

    # trial should return at least 60 neighbors to be used in GNNS
    # penalize model if slower than brute force
    c0 = - min_neighbors.value + 60
    c1 = average_time.value - 0.01
    trial.set_user_attr('constraint', (c0, c1))

    return maf.value, average_time.value

def constraints(trial):
    return trial.user_attrs['constraint']

In [4]:
%%time
for i in range(10):
    try:
        sampler = optuna.samplers.NSGAIISampler(constraints_func=constraints)
        lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize'], sampler=sampler)
        lsh_study.optimize(objective_lsh, n_trials=50)
        print("-------------------- Best trials --------------------")
        trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
        # print feasible trials only
        for trial in trials:
            print("Trial no. {}".format(trial.number))
            print(" Values = {}, Constraints = {}".format(trial.values, trial.user_attrs["constraint"]))
            print(" Params = {}".format(trial.params))
        break
    except:
        print("Trial failed, trying again...")
        continue

/home/thantrisp/miniconda3/envs/testpy/lib/python3.12/site-packages/optuna/samplers/nsgaii/_sampler.py:162: ExperimentalWarning: The constraints_func option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2023-11-26 16:14:04,282] A new study created in memory with name: lsh


Trial parameters: {'k': 10, 'L': 6, 'table_size': 15000, 'window_size': 2190, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 16:14:52,662] Trial 0 finished with values: [1.4737879905085032, 0.0029157] and parameters: {'k': 10, 'L': 6, 'table_size': 15000, 'window_size': 2190, 'query_trick': True}. 


Trial parameters: {'k': 3, 'L': 3, 'table_size': 3750, 'window_size': 1063, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 16:15:23,545] Trial 1 finished with values: [1.499912106564422, 0.00921434] and parameters: {'k': 3, 'L': 3, 'table_size': 3750, 'window_size': 1063, 'query_trick': False}. 


...

Trial parameters: {'k': 8, 'L': 9, 'table_size': 15000, 'window_size': 2242, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 16:55:00,642] Trial 48 finished with values: [1.152395647747845, 0.010039130000000002] and parameters: {'k': 8, 'L': 9, 'table_size': 15000, 'window_size': 2242, 'query_trick': False}. 


Trial parameters: {'k': 3, 'L': 5, 'table_size': 7500, 'window_size': 2881, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query: 

[I 2023-11-26 16:55:50,453] Trial 49 finished with values: [1.0152371815700856, 0.14100393999999997] and parameters: {'k': 3, 'L': 5, 'table_size': 7500, 'window_size': 2881, 'query_trick': True}. 


-------------------- Best trials --------------------
Trial no. 31
 Values = [1.0, 0.15069452999999997], Constraints = (0, 0.14069452999999996)
 Params = {'k': 6, 'L': 10, 'table_size': 3750, 'window_size': 3677, 'query_trick': True}
Trial no. 2
 Values = [1.0016436954345849, 0.12031295999999997], Constraints = (0, 0.11031295999999997)
 Params = {'k': 4, 'L': 8, 'table_size': 3750, 'window_size': 2921, 'query_trick': True}
Trial no. 20
 Values = [1.0019077119803148, 0.09559584999999998], Constraints = (0, 0.08559584999999999)
 Params = {'k': 5, 'L': 7, 'table_size': 3750, 'window_size': 2805, 'query_trick': True}
Trial no. 9
 Values = [1.01098321046841, 0.07159987000000002], Constraints = (0, 0.06159987000000002)
 Params = {'k': 9, 'L': 6, 'table_size': 7500, 'window_size': 4659, 'query_trick': True}
Trial no. 47
 Values = [1.0157488177897585, 0.06259387], Constraints = (0, 0.052593869999999994)
 Params = {'k': 7, 'L': 8, 'table_size': 3750, 'window_size': 3304, 'query_trick': False}
T

In [5]:
plot_pareto_front(lsh_study, target_names=['MAF', 'Average time'])

In [6]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'MAF')

In [7]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'Average time')

In [8]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'MAF')

In [9]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'Average time')

# Optimize without constraints

In [3]:
def objective_lsh(trial):
    param_dict = {'k': trial.suggest_int('k', 2, 10),
                  'L': trial.suggest_int('L', 2, 10),
                  'table_size': trial.suggest_categorical('table_size', [int(n/16), int(n/8), int(n/4)]),
                  'window_size': trial.suggest_int('window_size', 1000, 5000),
                  'query_trick': trial.suggest_categorical('query_trick', [True, False])
                 }
    
    print("Trial parameters:", param_dict)

    average_time, maf, min_neighbors = lsh_test(input_path, query_path, queries_num=100, **param_dict, N=50)

    return maf.value, average_time.value, min_neighbors.value

In [4]:
%%time
lsh_study = optuna.create_study(study_name='lsh', directions=['minimize', 'minimize', 'maximize'])
lsh_study.optimize(objective_lsh, n_trials=50)

[I 2023-11-26 18:29:54,478] A new study created in memory with name: lsh


Trial parameters: {'k': 9, 'L': 6, 'table_size': 15000, 'window_size': 2456, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 18:30:38,632] Trial 0 finished with values: [1.2863161135729677, 0.004823560000000001, 50.0] and parameters: {'k': 9, 'L': 6, 'table_size': 15000, 'window_size': 2456, 'query_trick': True}. 


Trial parameters: {'k': 2, 'L': 7, 'table_size': 3750, 'window_size': 1288, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 18:31:17,262] Trial 1 finished with values: [1.027435474263356, 0.07817652000000001, 50.0] and parameters: {'k': 2, 'L': 7, 'table_size': 3750, 'window_size': 1288, 'query_trick': False}. 


...

Trial parameters: {'k': 5, 'L': 9, 'table_size': 15000, 'window_size': 3438, 'query_trick': False}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query

[I 2023-11-26 19:13:49,792] Trial 48 finished with values: [1.0, 0.22487005000000007, 50.0] and parameters: {'k': 5, 'L': 9, 'table_size': 15000, 'window_size': 3438, 'query_trick': False}. 


Trial parameters: {'k': 3, 'L': 4, 'table_size': 15000, 'window_size': 3182, 'query_trick': True}
Read MNIST data
Done
Algorithm: LSH
Query: 0
Query: 1
Query: 2
Query: 3
Query: 4
Query: 5
Query: 6
Query: 7
Query: 8
Query: 9
Query: 10
Query: 11
Query: 12
Query: 13
Query: 14
Query: 15
Query: 16
Query: 17
Query: 18
Query: 19
Query: 20
Query: 21
Query: 22
Query: 23
Query: 24
Query: 25
Query: 26
Query: 27
Query: 28
Query: 29
Query: 30
Query: 31
Query: 32
Query: 33
Query: 34
Query: 35
Query: 36
Query: 37
Query: 38
Query: 39
Query: 40
Query: 41
Query: 42
Query: 43
Query: 44
Query: 45
Query: 46
Query: 47
Query: 48
Query: 49
Query: 50
Query: 51
Query: 52
Query: 53
Query: 54
Query: 55
Query: 56
Query: 57
Query: 58
Query: 59
Query: 60
Query: 61
Query: 62
Query: 63
Query: 64
Query: 65
Query: 66
Query: 67
Query: 68
Query: 69
Query: 70
Query: 71
Query: 72
Query: 73
Query: 74
Query: 75
Query: 76
Query: 77
Query: 78
Query: 79
Query: 80
Query: 81
Query: 82
Query: 83
Query: 84
Query: 85
Query: 86
Query:

[I 2023-11-26 19:14:44,072] Trial 49 finished with values: [1.003079571761728, 0.14983422000000002, 50.0] and parameters: {'k': 3, 'L': 4, 'table_size': 15000, 'window_size': 3182, 'query_trick': True}. 


CPU times: user 34min 5s, sys: 10min 39s, total: 44min 44s
Wall time: 44min 49s


In [5]:
print("-------------------- Best trials --------------------")
trials = sorted(lsh_study.best_trials, key=lambda x: x.values)
# print feasible trials only
for trial in trials:
    print("Trial no. {}".format(trial.number))
    print(" Values = {}".format(trial.values))
    print(" Params = {}".format(trial.params))


-------------------- Best trials --------------------
Trial no. 18
 Values = [1.0, 0.13765656999999992, 50.0]
 Params = {'k': 8, 'L': 10, 'table_size': 7500, 'window_size': 4570, 'query_trick': True}
Trial no. 23
 Values = [1.0000329705767328, 0.07704273000000002, 50.0]
 Params = {'k': 10, 'L': 9, 'table_size': 15000, 'window_size': 4428, 'query_trick': False}
Trial no. 13
 Values = [1.0017920583849802, 0.07380007999999999, 50.0]
 Params = {'k': 10, 'L': 10, 'table_size': 3750, 'window_size': 4989, 'query_trick': False}
Trial no. 29
 Values = [1.0064591950958244, 0.061548800000000015, 50.0]
 Params = {'k': 7, 'L': 9, 'table_size': 3750, 'window_size': 3629, 'query_trick': False}
Trial no. 11
 Values = [1.014306164419062, 0.05221652000000002, 50.0]
 Params = {'k': 8, 'L': 10, 'table_size': 7500, 'window_size': 3824, 'query_trick': True}
Trial no. 36
 Values = [1.0309397645013207, 0.03882437999999999, 50.0]
 Params = {'k': 7, 'L': 10, 'table_size': 15000, 'window_size': 2657, 'query_tric

In [6]:
plot_pareto_front(lsh_study, target_names=['maf', 'average_time', 'min_neighbors'])

In [7]:
plot_optimization_history(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [8]:
plot_optimization_history(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [9]:
plot_optimization_history(lsh_study, target = lambda t: t.values[2], target_name = 'min_neighbors')

In [10]:
plot_slice(lsh_study, target = lambda t: t.values[0], target_name = 'maf')

In [11]:
plot_slice(lsh_study, target = lambda t: t.values[1], target_name = 'average_time')

In [12]:
plot_optimization_history(lsh_study, target = lambda t: t.values[2], target_name = 'min_neighbors')